In [2]:
from operator import itemgetter
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI


# ChatOpenAI 모델을 초기화합니다.
model = ChatOpenAI()
# 대화형 프롬프트를 생성합니다. 이 프롬프트는 시스템 메시지, 이전 대화 내역, 그리고 사용자 입력을 포함합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

In [3]:
# 대화 메모리 생성
memory = ConversationBufferMemory(
    return_messages=True,
    memory_key="chat_history"
)

# 메모리 초기화 
memory.load_memory_variables({})

{'chat_history': []}

In [4]:
# runnable 생성
runnable = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("chat_history")
)

In [5]:
# chain 생성
chain = runnable | prompt | model 

In [6]:
question = "안녕? 내 이름은 나래야."
response = chain.invoke({"input": question})
print(response)

content='안녕하세요, 나래님! 만나서 반가워요. 무엇을 도와드릴까요?' response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 29, 'total_tokens': 64}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-6562652f-594f-4e9c-804c-45f08d9e0ab1-0'


In [7]:
# 메모리 저장하기 
memory.save_context(
    {"input": question}, {"output": response.content}
)

# 메모리 출력
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='안녕? 내 이름은 나래야.'),
  AIMessage(content='안녕하세요, 나래님! 만나서 반가워요. 무엇을 도와드릴까요?')]}

In [8]:
question = "내 이름을 기억해?"
response = chain.invoke({"input": question})
print(response)

content='네, 알겠어요. 당신의 이름은 나래입니다. 어떤 도움이 필요하신가요?' response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 80, 'total_tokens': 117}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-51c78dd2-d77d-4ac4-afb0-e8e7b3860db0-0'


---

In [9]:
question = "나는 PPS라는 회사에 다니고 있어"
response = chain.invoke({"input": question})
print(response)

content='PPS라는 회사에서 일하고 계시다니 멋있네요! 업무나 직무 관련해서 도와드릴 내용이 있나요?' response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 86, 'total_tokens': 134}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-eed27b9d-c406-4eb3-8f1e-4755794ff913-0'
